# Headphones v Speakers Motion Capture Data

## Installing and importing libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib

%matplotlib inline

import matplotlib.pyplot as plt

import math

import sys

import pylab

import numpy.linalg

import plotly.plotly as py

%load_ext rpy2.ipython

#r packages
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector
stats = importr('stats')
base = importr('base')

from scipy import stats

import seaborn

from IPython.display import display, Markdown

#pyrqa packages
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.computing_type import ComputingType
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation

## Reading MoCap csv files for all participants

In [ ]:
#Reading csv files



hp_raw_mocap = {} #headphones condition

path = r'/Users/vesanche/Desktop/hp/' # use your path for the folder containing Headphones mocap files
hp_files = sorted(glob.glob(os.path.join(path, "*.csv")))

i=1

for filename in hp_files:
    hp_raw_mocap[i] = pd.read_csv(filename, delimiter=',', encoding='utf-8', skiprows=10, low_memory=False, na_values='0')
    i = i+1

In [ ]:
sp_raw_mocap = {} #speakers condition

path = r'/Users/vesanche/Desktop/sp/' # use your path for the folder containing Speakers mocap files
sp_files = sorted(glob.glob(os.path.join(path, "*.csv")))

i=1

for filename in sp_files:
    sp_raw_mocap[i] = pd.read_csv(filename, delimiter=',', encoding='utf-8', skiprows=10, low_memory=False, na_values='0')
    i = i+1


In [ ]:
hp_raw_mocap[35]

In [ ]:
#Inserting column with time for all participants in both conditions

for j in range (1,len(hp_raw_mocap)+1):
  time = np.linspace(0, len(hp_raw_mocap[j])/200, len(hp_raw_mocap[j]))
  hp_raw_mocap[j].insert(0, 'Time (s)', time)
  
for i in range (1,len(sp_raw_mocap)+1):
  time = np.linspace(0, len(sp_raw_mocap[i])/200, len(sp_raw_mocap[i]))
  sp_raw_mocap[i].insert(0, 'Time (s)', time)

# Preprocessing data

## Calculating velocities (QoM) for all markers, all participants

In [ ]:
#Function that computes vector norm of displacement
def displacement(posdata):
    
    times = np.linspace(0, len(posdata)/200, len(posdata))
    posdata.insert(0, 'Time (s)', times)

    displacement = (
        np.roll(posdata, -1, axis=0)
        - posdata)[:-1]
    
    #displacement = displacement.drop(displacement.index[len(displacement)-1])
    

    displacement = np.sqrt(
         displacement.iloc[:,1] ** 2 +
         displacement.iloc[:,2] ** 2 +
         displacement.iloc[:,3] ** 2
    )

    return pd.DataFrame({
        'Time (s)': posdata['Time (s)'][:-1],
        'Distance': displacement,
    })

In [ ]:
#Function that computes vector norm of velocity

def velocity(posdata):
  
  
    times = np.linspace(0, len(posdata)/200, len(posdata))
    posdata.insert(0, 'Time (s)', times)
    
    displacement = (
        np.roll(posdata, -1, axis=0)
        - posdata)[:-1]
    
    #displacement = displacement.drop(displacement.index[len(displacement)-1])
    

    speeds = np.sqrt(
         displacement.iloc[:,1] ** 2 +
         displacement.iloc[:,2] ** 2 +
         displacement.iloc[:,3] ** 2
    ) /  displacement['Time (s)']

    return pd.DataFrame({
        'Time (s)': posdata['Time (s)'][:-1],
        'Velocity': speeds,
    })

In [ ]:
#Running displacement function for all participants - HP condition
hp_displacements_mocap = {}    

for participant in range (1,len(hp_raw_mocap)+1):
    hp_displacements_mocap[participant] = {}
    
    j = 0
    k = 1
    
    for markers in range (4,len(hp_raw_mocap[participant].columns)+1,3):
      j=j+1
      hp_displacements_mocap[participant][j] = displacement(hp_raw_mocap[participant].iloc[:,k:markers:1])
      k = k+3
      hp_displacements_mocap[participant][j].index =  hp_displacements_mocap[participant][j]['Time (s)']
      hp_displacements_mocap[participant][j] =  hp_displacements_mocap[participant][j].drop(['Time (s)'], axis=1)

In [ ]:
#Running displacement function for all participants - SP condition
sp_displacements_mocap = {}    

for participant in range (1,len(sp_raw_mocap)+1):
    sp_displacements_mocap[participant] = {}
    
    j = 0
    k = 1

    for markers in range (4,len(sp_raw_mocap[participant].columns)+1,3): 
      j=j+1
      sp_displacements_mocap[participant][j] = displacement(sp_raw_mocap[participant].iloc[:,k:markers:1])
      k = k+3
      sp_displacements_mocap[participant][j].index =  sp_displacements_mocap[participant][j]['Time (s)']
      sp_displacements_mocap[participant][j] =  sp_displacements_mocap[participant][j].drop(['Time (s)'], axis=1)

In [ ]:
sp_displacements_mocap[1][5] #Displacement from participant 1, marker 21 - SP condition

In [ ]:
#Running velocity (QoM) function for all participants - HP condition
hp_velocities_mocap = {}    

for participant in range (1,len(hp_raw_mocap)+1):
    hp_velocities_mocap[participant] = {}
    
    j = 0
    k = 1
    
    for markers in range (4,len(hp_raw_mocap[participant].columns)+1,3):
      j=j+1
      hp_velocities_mocap[participant][j] = velocity(hp_raw_mocap[participant].iloc[:,k:markers:1])
      k = k+3
      hp_velocities_mocap[participant][j].index =  hp_velocities_mocap[participant][j]['Time (s)']
      hp_velocities_mocap[participant][j] =  hp_velocities_mocap[participant][j].drop(['Time (s)'], axis=1)

In [ ]:
#Running velocity (QoM) function for all participants - SP condition
sp_velocities_mocap = {}    

for participant in range (1,len(sp_raw_mocap)+1):
    sp_velocities_mocap[participant] = {}
    
    j = 0
    k = 1
    
    for markers in range (4,len(sp_raw_mocap[participant].columns)+1,3):
      j=j+1
      sp_velocities_mocap[participant][j] = velocity(sp_raw_mocap[participant].iloc[:,k:markers:1])
      k = k+3
      sp_velocities_mocap[participant][j].index =  sp_velocities_mocap[participant][j]['Time (s)']
      sp_velocities_mocap[participant][j] =  sp_velocities_mocap[participant][j].drop(['Time (s)'], axis=1)

In [ ]:
sp_velocities_mocap[1][1]

In [ ]:
#Concatenating velocity from all markers for each participant - HP condition
hp_vel_markers_mocap = {}
for participant in range (1,len(hp_raw_mocap)+1):
    hp_vel_markers_mocap[participant] = pd.concat(hp_velocities_mocap[participant],1)

In [ ]:
#Concatenating velocity from all markers for each participant - SP condition
sp_vel_markers_mocap = {}
for participant in range (1,len(sp_raw_mocap)+1):
    sp_vel_markers_mocap[participant] = pd.concat(sp_velocities_mocap[participant],1)

In [ ]:
#Assigning markers' names according to labelling

marker_labels_35hp = ["HF", "HL", "BU", "COM", "CHEST", "SHL", "SHR", "ELL", "ELR", "HIPL", "HIPR", "HANDL", "HANDR", "KNL", "KNR", "HEELL", "HEELR", "TOEL", "TOER", "BOARD", "FLOOR"]
marker_labels_3hp = ["HL", "HR", "BU", "COM", "CHEST", "SHL", "SHR", "ELL", "ELR", "HIPL", "HIPR", "HANDL", "HANDR", "KNL", "KNR", "HEELL", "HEELR", "TOEL", "TOER", "BOARD", "FLOOR"]
marker_labels =  ["HF", "HL", "HR", "BU", "COM", "CHEST", "SHL", "SHR", "ELL", "ELR", "HIPL", "HIPR", "HANDL", "HANDR", "KNL", "KNR", "HEELL", "HEELR", "TOEL", "TOER", "BOARD", "FLOOR"]
marker_labels_916sp =  ["HL", "HR", "BU", "COM", "CHEST", "SHL", "SHR", "ELL", "ELR", "HIPL", "HIPR", "HANDL", "HANDR", "KNL", "KNR", "HEELL", "HEELR", "TOEL", "TOER", "BOARD", "FLOOR"]
marker_labels_1sp =  ["HF", "HL", "HR", "BU", "COM", "CHEST", "SHL", "SHR", "ELL", "ELR", "HIPL", "HIPR", "HANDL", "HANDR", "KNL", "KNR", "HEELR", "TOEL", "TOER", "BOARD", "FLOOR"]

hp_vel_markers_mocap[35].columns = marker_labels_35hp
hp_vel_markers_mocap[3].columns = marker_labels_3hp
sp_vel_markers_mocap[9].columns = marker_labels_916sp
sp_vel_markers_mocap[16].columns = marker_labels_916sp
sp_vel_markers_mocap[1].columns = marker_labels_1sp


for participant in range (1,3):
    hp_vel_markers_mocap[participant].columns = marker_labels
for participant in range (4,35):
    hp_vel_markers_mocap[participant].columns = marker_labels
for participant in range (36,43):
    hp_vel_markers_mocap[participant].columns = marker_labels
    
for participant in range (2,9):
    sp_vel_markers_mocap[participant].columns = marker_labels
for participant in range (10,16):
    sp_vel_markers_mocap[participant].columns = marker_labels
for participant in range (17,43):
    sp_vel_markers_mocap[participant].columns = marker_labels

for participant in range (1,len(hp_raw_mocap)+1):
    hp_vel_markers_mocap[participant] =  hp_vel_markers_mocap[participant].drop(["BOARD", "FLOOR"], axis=1) #removing floor and wiiboard markers data before further analysis
    
for participant in range (1,len(sp_raw_mocap)+1):
    sp_vel_markers_mocap[participant] =  sp_vel_markers_mocap[participant].drop(["BOARD", "FLOOR"], axis=1)
    
sp_vel_markers_mocap[1] #velocity, all markers participant 1, speakers condition

In [ ]:
hp_vel_markers_mocap[35]

## Calculating mean of velocity for all markers, all participants

In [ ]:
for participant in range (1,len(sp_raw_mocap)+1):
  sp_vel_markers_mocap[participant]['mean'] = sp_vel_markers_mocap[participant][sp_vel_markers_mocap[participant].columns].mean(axis=1)
  sp_vel_markers_mocap[participant].loc['mean'] = sp_vel_markers_mocap[participant].mean()
  
  
for participant in range (1,len(hp_raw_mocap)+1):
  hp_vel_markers_mocap[participant]['mean'] = hp_vel_markers_mocap[participant][hp_vel_markers_mocap[participant].columns].mean(axis=1)
  hp_vel_markers_mocap[participant].loc['mean'] = hp_vel_markers_mocap[participant].mean()
  

In [ ]:
hp_vel_markers_mocap[1] #velocity and mean velocity, all markers, participant 1

## Smoothing velocity (TO BE DONE)

In [ ]:
hp_smooth_emg = {}

for i in list(hp_raw_emg):
        hp_smooth_emg[i] = hp_rectfd_emg[i].rolling(1000,center=True,min_periods=1).mean()

In [ ]:
sp_smooth_emg = {}

for i in list(sp_raw_emg):
        sp_smooth_emg[i] = sp_rectfd_emg[i].rolling(1000,center=True,min_periods=1).mean()

# Splitting movement data into conditions/songs

In [ ]:
hp_songorder = pd.read_csv('drive/My Drive/UiO RITMO/HeadphonesVSSpeakers/songorderhp.csv') #Reading csv with stimuli order used for each participant during headphones condition
hp_songorder.columns = ['Participant','1', '2', '3','4', '5', '6']
hp_songorder.set_index('Participant',inplace=True, drop=True)

In [ ]:
sp_songorder = pd.read_csv('drive/My Drive/UiO RITMO/HeadphonesVSSpeakers/songordersp.csv') #Reading csv with stimuli order used for each participant during speakers condition
sp_songorder.columns = ['Participant','1', '2', '3','4', '5', '6']
sp_songorder.set_index('Participant',inplace=True, drop=True)

## Segmenting velocity data into music-silence segments

In [ ]:
def segments(songorder,data):
    #work in progress, works for emg data at 200Hz, song duration used in 2018 headphones-speakers experiment
    #Silence 1 - 45?
    #Silence 2, 3, 4, 5, 6 - 30s
    #1 -  - 49s
    #2 -  - 48s
    #3 -  - 47s
    #4 -  - 48s
    #5 -  - 48s
    #6 -  - 48s
    #Silence 7 - 32s
    #Total duration of recording 500s
   
    
    start = {}
    segments = {}
    segments[1] = data[0:6000] #First silence segement: 30 seconds
    start[1] = 0
    start[2] = 6001
    song = 0
    
    while song <= len(songorder.columns)-1:
        for inicio in range (2,13,2):
            if songorder.iloc[0][song] == 1:
                        segments[inicio]=data[start[inicio]:start[inicio]+9800] #song1 49s
                        segments[inicio+1]=data[start[inicio]+9801:start[inicio]+9800+6000] #30s silence
                        start[inicio+2] = start[inicio]+9801+6000
                        song = song+1  
            elif songorder.iloc[0][song] == 2:
                        segments[inicio]=data[start[inicio]:start[inicio]+9600] #song2 48s
                        segments[inicio+1]=data[start[inicio]+9601:start[inicio]+9600+6000] #30s silence
                        start[inicio+2] = start[inicio]+9601+6000
                        song = song+1
            elif songorder.iloc[0][song] == 3:
                        segments[inicio]=data[start[inicio]:start[inicio]+9400] #song3 47s
                        segments[inicio+1]=data[start[inicio]+9401:start[inicio]+9400+6000] #30s silence
                        start[inicio+2] = start[inicio]+9401+6000
                        song = song+1
            elif songorder.iloc[0][song] == 4:
                        segments[inicio]=data[start[inicio]:start[inicio]+9600] #song4 48s
                        segments[inicio+1]=data[start[inicio]+9601:start[inicio]+9600+6000] #30s silence
                        start[inicio+2] = start[inicio]+9601+6000
                        song = song+1
            elif songorder.iloc[0][song] == 5:
                        segments[inicio]=data[start[inicio]:start[inicio]+9600] #song5 48s
                        segments[inicio+1]=data[start[inicio]+9601:start[inicio]+9600+6000] #30s silence
                        start[inicio+2] = start[inicio]+9601+6000
                        song = song+1
            elif songorder.iloc[0][song] == 6:
                        segments[inicio]=data[start[inicio]:start[inicio]+9600] #song6 48s
                        segments[inicio+1]=data[start[inicio]+96001:start[inicio]+9600+6000] #silence
                        start[inicio+2] = start[inicio]+9601+6000
                        song = song+1
    return segments

In [ ]:
hp_vel_mocap_segm = {}  #Using segments function on headphones condition processed data for all participants
for i in list(hp_raw_mocap):
    hp_vel_mocap_segm[i] = segments(hp_songorder[i-1:i],hp_vel_markers_mocap[i])

In [ ]:
sp_vel_mocap_segm = {}  #Using segments function on headphones condition processed data for all participants
for i in list(sp_raw_mocap):
    sp_vel_mocap_segm[i] = segments(sp_songorder[i-1:i],sp_vel_markers_mocap[i])

## Grouping segmented data by music/silence condition

In [ ]:
#Odd segments are silence, even segments are music
hp_vel_mocap_segm_mus = {} #creating empty dictionaries for music and silence
hp_vel_mocap_segm_sil = {}

for i in list(hp_raw_mocap):
  hp_vel_mocap_segm_mus[i] = {} #creating empty dictionary for each participant for both music and silence conditions
  hp_vel_mocap_segm_sil[i] = {}
  k = 0
  
  for j in range (1,13,2):
    hp_vel_mocap_segm_sil[i][k] = hp_vel_mocap_segm[i][j] #filling each participant's dictionary with odd segments (silence)
    hp_vel_mocap_segm_mus[i][k] = hp_vel_mocap_segm[i][j+1] #filling each participant's dictionary with even segments (music)
    k=k+1
 
  

In [ ]:
#Same as above, for speakers condition
sp_vel_mocap_segm_mus = {} #creating empty dictionaries for music and silence
sp_vel_mocap_segm_sil = {}

for i in list(sp_raw_mocap):
  sp_vel_mocap_segm_mus[i] = {} #creating empty dictionary for each participant for both music and silence conditions
  sp_vel_mocap_segm_sil[i] = {}
  kx = 0
  
  for j in range (1,13,2):
    sp_vel_mocap_segm_sil[i][kx] = sp_vel_mocap_segm[i][j] #filling each participant's dictionary with odd segments (silence)
    sp_vel_mocap_segm_mus[i][kx] = sp_vel_mocap_segm[i][j+1] #filling each participant's dictionary with even segments (music)
    kx=kx+1
 

In [ ]:
sp_vel_mocap_segm_mus[3][0] #1st music segments participant 3, speakers condition

## Sorting music data by stimuli

In [ ]:
## Function to sort segments from each participant with corresponding stimuli
def sort_stimuli(songorder,data):
   
    sorted_stimuli = {}
    song = 0
    while song <= len(songorder.columns)-1:
        
            if songorder.iloc[0][song] == 1:
                       sorted_stimuli[1] = data[song]
                       song = song+1
            elif songorder.iloc[0][song] == 2:
                       sorted_stimuli[2] = data[song]
                       song = song+1
            elif songorder.iloc[0][song] == 3:
                       sorted_stimuli[3] = data[song]
                       song = song+1
            elif songorder.iloc[0][song] == 4:
                       sorted_stimuli[4] = data[song]
                       song = song+1
            elif songorder.iloc[0][song] == 5:
                       sorted_stimuli[5] = data[song]
                       song = song+1
            elif songorder.iloc[0][song] == 6:
                       sorted_stimuli[6] = data[song]
                       song = song+1
    return sorted_stimuli

In [ ]:
sp_songorder #Reminding song order for the headphones condition

In [ ]:
## Running stimuli-sorting function for all participants
hp_vel_mocap_sort_mus = {}
for i in list(hp_raw_mocap):  
    hp_vel_mocap_sort_mus[i] = sort_stimuli(hp_songorder[i-1:i],hp_vel_mocap_segm_mus[i])

In [ ]:
## Same as above, for speakers condition
sp_vel_mocap_sort_mus = {}
for i in list(sp_raw_mocap):  
    sp_vel_mocap_sort_mus[i] = sort_stimuli(sp_songorder[i-1:i],sp_vel_mocap_segm_mus[i])

In [ ]:
sp_vel_mocap_sort_mus[4][1] #velocity data, participant 4, song 1=Andre Bratten. Headphones condition